In [1]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint
from copy import deepcopy
import dill

In [2]:
## initial data needed to generate setup of rotors through Bombe Scrambler
# entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
# forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}
# rev_rotors = {}
# for r in raw_rotors.keys():
#     working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
#     rev_rotors[r] = [working[k] for k in sorted(working.keys())]
# reflectors = {'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 
#               'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}

entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
notches = {'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}
## forward rotors is the forward in:out pairings of each rotor as the character index of the A-Z ascii alphabet stored in 'entry'
forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}

## couldn't see a straightforward list comprehension for the reverse - the in:out pairing for when the current flows
## back from the reflector to the final output. Hopefully this two-step for loop isn't too slow
rev_rotors = {}
for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

# for k,v in rev_rotors.items():
#     print(k,':',v)

## in:out pairings for reflectors are hardcoded in, were originally generated by code
reflectors = {'B': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'C': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}
# print(reflectors)

In [3]:
class ScramblerBombe:
    
    def __init__(self,left_rotor,middle_rotor,right_rotor,reflector,menu_link='ZZZ',conx_in={},conx_out={}):
        """rotors must be strings referring to either ['I','II','III','IV','V']
        reflector must be string, one of either ['B','C']"""
        
        self.right_rotor = right_rotor
        self.middle_rotor = middle_rotor
        self.left_rotor = left_rotor
        self.reflector = reflectors[reflector]
        self.menu_link = menu_link
        self.middle_notch = entry.index(notches[self.middle_rotor])   ## point if right rotor reaches will trigger middle rotor to step
        self.left_notch = entry.index(notches[self.left_rotor])  ## point if middle rotor reaches will trigger left rotor to step
        self.pos_left_rotor, self.pos_mid_rotor, self.pos_rgt_rotor = (ascii_uppercase.index(m) for m in menu_link.upper())
        self.in_status = {char:0 for char in entry}
        self.out_status = {char:0 for char in entry}
        self.conx_in = conx_in
        self.conx_out = conx_out
        
    def once_thru_scramble(self,start_character, direction, first_rotor, pos1, second_rotor, pos2, 
                       third_rotor, pos3):
        """ start_character must be single ASCII character A-Z
        direction is either 'forward' or 'back' """
        if direction == 'forward':
            usedict = {k:v for k,v in forward_rotors.items()}
        elif direction == 'back':
            usedict = {k:v for k,v in rev_rotors.items()}
        else:
            print('only forward or back for direction')
            return 'wtf'

        start_character = start_character.upper()
        entry_pos = entry.index(start_character)
        fst_pos_modifier = (26 + pos1 - 0)%26
        fst_in = (entry_pos + fst_pos_modifier)%26
        fst_out = usedict[first_rotor][fst_in]
        ch1o = entry[fst_out]

        scd_pos_modifier = (26 + pos2 - pos1)%26
        scd_in = (fst_out + scd_pos_modifier)%26
        ch2i = entry[scd_in]
        scd_out = usedict[second_rotor][scd_in]
        ch2o = entry[scd_out]

        thd_pos_modifier = (26 + pos3 - pos2)%26
        thd_in = (scd_out + thd_pos_modifier)%26
        ch3i = entry[thd_in]
        thd_out = usedict[third_rotor][thd_in]
        ch3o = entry[thd_out]

#         if direction == 'forward':
#             print(f"{start_character} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
#         elif direction == 'back':
#             print(f"{start_character} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")

        return ch3o
    
    def full_scramble(self,in_ch):
        in_ch = in_ch.upper()
        left_rotor = self.left_rotor
        middle_rotor = self.middle_rotor
        right_rotor = self.right_rotor
        rflector = self.reflector
        ## first run right to left through scrambler
        forward_run = self.once_thru_scramble(in_ch, direction='forward', first_rotor=right_rotor, pos1=self.pos_rgt_rotor, 
                                              second_rotor=middle_rotor, pos2=self.pos_mid_rotor, 
                                              third_rotor=left_rotor, pos3=self.pos_left_rotor)

        ## reflector back around for return
        rfi_pos_mod = (26 + 0 - self.pos_left_rotor)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
        rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
        chri = entry[rf_in]
        mirrored = rflector[chri]

#         print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")

        ## second run back left to right thru scrambler
        back_run = self.once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=self.pos_left_rotor, 
                                      second_rotor=middle_rotor, pos2=self.pos_mid_rotor, third_rotor=right_rotor, pos3=self.pos_rgt_rotor)

        bk_out = entry.index(back_run)
        bko_pos_mod = (26 + 0 - self.pos_rgt_rotor)%26  ## as above, '0' just reflects that the entry interface doesn't move
        bk_final = (bk_out + bko_pos_mod)%26
        final = entry[bk_final]
#         print('RR back out:  ', back_run, '-->', final)
#         print(in_ch,"-->",final)
        return final
    
    def rotor_step(self,rotor_position):
        """"""
        if rotor_position == 25:
            rotor_position = 0
        else:
            rotor_position += 1
        return rotor_position
    
    def step_enigma(self):
        """Just acts on itself, steps the rotors"""
        
#         print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")
        
        if self.pos_rgt_rotor == self.middle_notch and self.pos_mid_rotor == self.left_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            self.pos_left_rotor = self.rotor_step(self.pos_left_rotor)
            print('--- left & middle rotor step')
        elif self.pos_rgt_rotor == self.middle_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            print('--- middle rotor step ---')
        else:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            
        print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")

    def update(self):
        """idea here is that the scrambler will check each of the 26 connections to see if they
        are live, and if so pass it through itself to live up the connection on the other side of 
        the scrambler"""
        
        for char, status in self.in_status.items():
            if status == 0:
                pass
            else:
                self.out_status[self.full_scramble(char)] = 1
                
        for char, status in self.out_status.items():
            if status == 0:
                pass
            else:
                self.in_status[self.full_scramble(char)] = 1
            


            

In [ ]:
# eb1 = ScramblerBombe(left_rotor='II',middle_rotor='V',right_rotor='III',reflector='C',menu_link='BCD')

In [ ]:
# eb1.in_status

In [ ]:
# eb1.in_status['E'] = 1
# eb1.in_status['H'] = 1
# eb1.in_status['P'] = 1
# eb1.in_status['X'] = 1
# eb1.in_status

In [ ]:
# eb1.out_status

In [ ]:
# eb1.update()
# eb1.out_status

In [ ]:
# %%time
# for i in range(26**3):
#     eb1.full_scramble('Q')
#     eb1.step_enigma()

In [ ]:
# word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [ ]:
# def only_ascii(instring):
#     """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
#     newstring = ''
#     for character in instring:
#         if character in ascii_letters:
#             newstring += character.upper()
#     return newstring

In [ ]:
# only_ascii(word)

In [125]:
class Bombe:
    
    def __init__(self,left_rotor='I',middle_rotor='II',right_rotor='III',reflector='B'):
        self.left_rotor = left_rotor
        self.middle_rotor = middle_rotor
        self.right_rotor = right_rotor
        self.reflector = reflector
        self.scramblers = {}
        self.register = {'status':{char:0 for char in entry}}
        
    def setup(self,menu):
        self.menu = menu
        for key,val in self.menu.items():
            if key == 'config':
                self.test_char = self.menu['config']['test_char']
                self.register['conxns'] = self.menu['config']['conx_in']
            else:
                self.scramblers[key] = ScramblerBombe(self.left_rotor,self.middle_rotor,self.right_rotor,self.reflector,
                                                      menu_link=val['menu_link'],conx_in=val['conx_in'], conx_out=val['conx_out'])
        
    
    def pulse_connections(self):
        """Goes through every scrambler (once only), and updates in and out terminals with live feeds via conx_in and
        conx_out from its connected scramblers. Should be run iteratively to exhaustion."""
        for order,scr in self.scramblers.items():
            ## i.e for each scrambler in the enigma, we're going to loop through it's conx_in and conx_out dict
            ## and check its connected scramblers to match any live wires
            for pos,side in scr.conx_in.items():
                ## for this scrambler, go through the conx_in dict. It has the other scrambler you need to 
                ## look at (pos, or the number/position label) and whether you're looking at the 'in' or 'out' side
                if side == 'in':
                    # for those conxns that are on the 'in' side
                    for ch,io in self.scramblers[pos].in_status.items():
                        # look through the scrambler's in_status
                        if io == 1:   # and if it has a live wire
                            scr.in_status[ch] == 1 # set the corresponding character in the conx_in dict of the 
                                                    # original scrambler being checked to also be live
                elif side == 'out':
                    for ch,io in self.scramblers[pos].out_status.items():
                        if io == 1:
                            scr.in_status[ch] == 1
                            
            for pos,side in scr.conx_out.items():   ## repeating as above for conx_out
                if side == 'in':
                    for ch,io in self.scramblers[pos].in_status.items():
                        if io == 1:
                            scr.out_status[ch] == 1
                elif side == 'out':
                    for ch,io in self.scramblers[pos].out_status.items():
                        if io == 1:
                            scr.out_status[ch] == 1
                
    def sync_test_register(self):
        
        for pos,side in self.register['conxns'].items():
            ## for each of the scrambler terminals connected to the test register
            if side == 'in':
                for ch,io in self.scramblers[pos].in_status.items():
                    ## first sync any live wires from the scrambler to the test register
                    if io == 1:   
                        self.register['status'][ch] = 1
                for ch,io in self.register['status'].items():
                    ## but then also sync any live wires from the test register to the scrambler
                    if io == 1:
#                         print(ch,io,pos)
                        self.scramblers[pos].in_status[ch] = 1
#                         print(self.scramblers[pos].in_status)
            
            elif side == 'out':
                for ch,io in self.scramblers[pos].out_status.items():
                    if io == 1:
                        self.register['status'][ch] = 1
                for ch,io in self.register['status'].items():
                    ## but then also sync any live wires from the test register to the scrambler
                    if io == 1:
                        self.scramblers[pos].out_status[ch] = 1
                        

    
    def light_character(self,in_character):
        """just need a way to put in the initial character input into all of the scramblers"""
#         for scr in self.scramblers.values():
#             scr.in_status[in_character] = 1
        self.register['status'][self.test_char] = 1
    
    def update_all(self):
        """For every scrambler, runs update() which passes live terminals through the scrambler - from in to out
        and vice versa, for whatever the current position is"""
        for scr in self.scramblers.keys():
            self.scramblers[scr].update()
            
    def spin_scramblers(self):
        """Runs step_enigma for all scramblers, spinning the right rotor once and perhaps the middle and left if
        they are in their notch positions"""
        for scr in self.scramblers.values():
            scr.step_enigma()

In [126]:
with open('menu_transfer.pickle', 'rb') as infile:
    menu = dill.load(infile)
menu

{4: {'in': 'A',
  'out': 'F',
  'menu_link': 'ZZD',
  'conx_in': {6: 'out', 9: 'out', 17: 'in'},
  'conx_out': {13: 'in'}},
 13: {'in': 'F',
  'out': 'D',
  'menu_link': 'ZZM',
  'conx_in': {4: 'out'},
  'conx_out': {16: 'in'}},
 16: {'in': 'D',
  'out': 'C',
  'menu_link': 'ZZP',
  'conx_in': {13: 'out'},
  'conx_out': {6: 'in'}},
 6: {'in': 'C',
  'out': 'A',
  'menu_link': 'ZZF',
  'conx_in': {16: 'out'},
  'conx_out': {4: 'in', 9: 'out', 17: 'in'}},
 2: {'in': 'Q',
  'out': 'W',
  'menu_link': 'ZZB',
  'conx_in': {14: 'out'},
  'conx_out': {}},
 14: {'in': 'U',
  'out': 'Q',
  'menu_link': 'ZZN',
  'conx_in': {10: 'out'},
  'conx_out': {2: 'in'}},
 10: {'in': 'J',
  'out': 'U',
  'menu_link': 'ZZJ',
  'conx_in': {11: 'out'},
  'conx_out': {14: 'in'}},
 11: {'in': 'T',
  'out': 'J',
  'menu_link': 'ZZK',
  'conx_in': {9: 'in', 5: 'in'},
  'conx_out': {10: 'in'}},
 9: {'in': 'T',
  'out': 'A',
  'menu_link': 'ZZI',
  'conx_in': {11: 'in', 5: 'in'},
  'conx_out': {4: 'in', 6: 'out', 1

In [127]:
tb1 = Bombe('I','II','III','B')

In [128]:
tb1.setup(menu)

In [129]:
tb1.light_character('A')
print(tb1.register['status'])

{'A': 1, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0}


In [135]:
tb1.update_all()
tb1.pulse_connections()
tb1.sync_test_register()
tb1.register['status']

{'A': 1,
 'B': 0,
 'C': 0,
 'D': 0,
 'E': 0,
 'F': 0,
 'G': 0,
 'H': 0,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 0,
 'O': 0,
 'P': 0,
 'Q': 0,
 'R': 0,
 'S': 0,
 'T': 0,
 'U': 0,
 'V': 0,
 'W': 0,
 'X': 0,
 'Y': 0,
 'Z': 0}

In [136]:
for scr in tb1.scramblers.values():
    print(scr.in_status)

{'A': 1, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0}
{'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0}
{'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0}
{'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 1, 'R': 0, 'S': 0, 'T': 0, 'U': 0, 'V': 0, 'W': 0, 'X': 0, 'Y': 0, 'Z': 0}
{'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0, 'I': 0, 'J': 0, 'K': 0, 'L': 0, 'M': 0, 'N': 0, 'O': 0, 'P': 0, 'Q': 0, 'R': 0, 'S': 0, 'T': 0, 'U'

In [138]:
for order,scr in tb1.scramblers.items():
    print(order, [k for k,v in scr.in_status.items() if v == 1], [k for k,v in scr.out_status.items() if v == 1])

4 ['A'] ['Y']
13 [] []
16 [] []
6 ['Q'] ['A']
2 [] []
14 [] []
10 [] []
11 [] []
9 ['W'] ['A']
5 [] []
17 ['A'] ['Z']
0 [] []


In [16]:
tb1.spin_scramblers()
tb1.update_all()
tb1.pulse_connections()

--- middle rotor step ---
Lpos=25, Mpos=0, Rpos=5
Lpos=25, Mpos=25, Rpos=14
Lpos=25, Mpos=25, Rpos=17
Lpos=25, Mpos=25, Rpos=7
Lpos=25, Mpos=25, Rpos=3
Lpos=25, Mpos=25, Rpos=15
Lpos=25, Mpos=25, Rpos=11
Lpos=25, Mpos=25, Rpos=12
Lpos=25, Mpos=25, Rpos=10
Lpos=25, Mpos=0, Rpos=6
Lpos=25, Mpos=25, Rpos=18
Lpos=25, Mpos=25, Rpos=1


In [17]:
for scr in tb1.scramblers.values():
    print([k for k,v in scr.in_status.items() if v == 1], [k for k,v in scr.out_status.items() if v == 1])

['A', 'J', 'X', 'Y'] ['G', 'L', 'X', 'Y']
['A', 'M', 'Z'] ['U', 'V', 'Y']
['A', 'J', 'S', 'X'] ['F', 'H', 'T', 'Z']
['A', 'I', 'P', 'T'] ['C', 'Q', 'U', 'V']
['A', 'U', 'V', 'Z'] ['L', 'T', 'W', 'Y']
['A', 'L', 'S', 'Z'] ['F', 'J', 'U', 'V']
['A', 'D', 'Q', 'X'] ['B', 'G', 'M', 'O']
['A', 'Q', 'R', 'Z'] ['B', 'O', 'U', 'Y']
['A', 'D', 'E', 'K'] ['B', 'M', 'P', 'W']
['A', 'G', 'O', 'Y'] ['L', 'P', 'S', 'X']
['A', 'F', 'P', 'S'] ['C', 'H', 'Q', 'Z']
['A', 'O', 'Q', 'Z'] ['B', 'E', 'W', 'X']


In [18]:
for scr in tb1.scramblers.values():
    print(scr.full_scramble('A'))

L
U
H
C
Y
F
O
Y
B
S
C
W
